<a href="https://colab.research.google.com/github/Weimw/movielens-project/blob/main/cs224w_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Weimw/movielens-project GitRepo

# Set up working directory
import os
wdir = 'GitRepo'
os.chdir(wdir)

Cloning into 'GitRepo'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 43 (delta 6), reused 37 (delta 6), pack-reused 0
Unpacking objects: 100% (43/43), 3.75 MiB | 4.92 MiB/s, done.


In [ ]:
!ls

data  ml-latest-small  project-colab.ipynb  README.md


In [ ]:
import sklearn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import networkx as nx

import data_utils

In [ ]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
data_utils.load_data('ml_100k')

Loading dataset ml_100k


TypeError: ignored